# Random Forest Prediction Model 

This notebook trains and evaluates an **optimized Random Forest model** to predict
future Yelp rating drops for Philadelphia restaurants.

It includes:
- Upgraded label definition based on **future 7-day rating decline**
- Enhanced feature engineering (+13 new features)
- SMOTE for class imbalance
- Threshold optimization (maximize F1-score)
- Baseline time-series model
- Model comparison across multiple algorithms
- SHAP-based model explainability
- Saving all model artifacts for downstream dashboards and reports


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE

import shap

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True

os.makedirs("models", exist_ok=True)
os.makedirs("figures", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)

print("Libraries imported.")


Libraries imported.


## 1. Load Feature-Engineered Data

We start from `reviews_features.csv`, created in **Notebook 1 (Data Prep)**.
This file already contains rolling 7-day rating and sentiment metrics.


In [ ]:
data_path = "data/processed/reviews_features.csv"
df = pd.read_csv(data_path, parse_dates=["date"])

print("Loaded data from:", data_path)
print("Shape:", df.shape)
print(df[["business_id", "date"]].head())


Loaded data from: data/processed/reviews_features.csv
Shape: (99997, 27)
              business_id                date
0  -0M0b-XhtFagyLmsBtOe8w 2012-02-21 18:24:45
1  -0M0b-XhtFagyLmsBtOe8w 2013-12-03 23:56:37
2  -0M0b-XhtFagyLmsBtOe8w 2015-06-24 22:10:39
3  -0PN_KFPtbnLQZEeb23XiA 2011-11-23 22:10:01
4  -0TffRSXXIlBYVbb5AwfTg 2013-06-01 01:47:50


## 2. Upgrade Label: Future 7-Day Rating Drop

We define a rating drop event as:

> The **future 7-day average rating** is at least 0.3 stars lower than the
> current 7-day average rating for the same business.

This gives us a business-meaningful target:
whether the restaurant is about to experience a noticeable decline.


In [ ]:
print("\n" + "=" * 70)
print("UPGRADING LABEL — FUTURE 7-DAY RATING DROP")
print("=" * 70)

# Sort by business and date to ensure temporal order
df = df.sort_values(["business_id", "date"])

if "rating_7d" not in df.columns:
    raise KeyError(
        "Column 'rating_7d' not found. Make sure Notebook 1 created this feature."
    )

# Compute future 7-day average rating (shift -7 within each business)
df["rating_7d_future"] = df.groupby("business_id")["rating_7d"].shift(-7)

drop_threshold = 0.3
df["label_rating_drop"] = (
    df["rating_7d_future"] <= (df["rating_7d"] - drop_threshold)
).astype(int)

print("Label distribution (proportion):")
print(df["label_rating_drop"].value_counts(normalize=True).rename("proportion"))



UPGRADING LABEL — FUTURE 7-DAY RATING DROP
Label distribution (proportion):
label_rating_drop
0    0.802824
1    0.197176
Name: proportion, dtype: float64


In [ ]:
# Drop rows with missing label or key features
initial_shape = df.shape
df = df.dropna(subset=["label_rating_drop", "rating_7d", "sentiment_7d"])
print(f"Dropped {initial_shape[0] - df.shape[0]} rows due to NaNs in key fields.")


Dropped 10119 rows due to NaNs in key fields.


## 3. Enhanced Feature Engineering

We add 13 new features on top of the base features from Notebook 1:
- Interaction features
- Time-based features
- Business-specific features
- Acceleration (second derivative) features


In [ ]:
print("\n" + "=" * 70)
print("PRIORITY 3: ENHANCED FEATURE ENGINEERING")
print("=" * 70)

# 1. Interaction features
print("\nCreating interaction features...")
df["rating_sentiment_interaction"] = df["rating_7d"] * df["sentiment_7d"]
df["momentum_volatility"] = df["sentiment_momentum"] * df["rating_volatility"]
print("✓ Interaction features created")

# 2. Time-based features
print("\nCreating time-based features...")
df["day_of_week"] = df["date"].dt.dayofweek
df["month"] = df["date"].dt.month
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)
df["is_holiday_season"] = df["month"].isin([11, 12]).astype(int)  # Nov-Dec
print("✓ Time-based features created")

# 3. Business-specific features
print("\nCreating business-specific features...")
df["review_density"] = df["review_count_7d"] / 7.0  # Reviews per day

# Negative review ratio in last 7 days
df["negative_ratio_7d"] = df.groupby("business_id")["stars_review"].transform(
    lambda s: s.le(2).rolling(window=7, min_periods=1).mean()
)

# Rating range (max - min) in last 7 days
df["rating_range_7d"] = df.groupby("business_id")["stars_review"].transform(
    lambda s: s.rolling(7, min_periods=3).max() - s.rolling(7, min_periods=3).min()
)

# Sentiment range in last 7 days
df["sentiment_range_7d"] = df.groupby("business_id")["sentiment"].transform(
    lambda s: s.rolling(7, min_periods=3).max() - s.rolling(7, min_periods=3).min()
)

print("✓ Business-specific features created")

# 4. Acceleration features (second derivative)
print("\nCreating acceleration features...")
df["sentiment_acceleration"] = df.groupby("business_id")[
    "sentiment_momentum"
].transform(lambda x: x.diff())
df["rating_acceleration"] = df.groupby("business_id")["rating_momentum"].transform(
    lambda x: x.diff()
)
print("✓ Acceleration features created")

print("\nEnhanced feature engineering complete.")



PRIORITY 3: ENHANCED FEATURE ENGINEERING

Creating interaction features...
✓ Interaction features created

Creating time-based features...
✓ Time-based features created

Creating business-specific features...
✓ Business-specific features created

Creating acceleration features...
✓ Acceleration features created

Enhanced feature engineering complete.


## 4. Train-Test Split and SMOTE

We now select all feature columns, build `X` and `y`, split into training and
test sets, and apply SMOTE to handle class imbalance.


In [ ]:
# === 4. Train-Test Split and SMOTE (numeric features only) ===

target_col = "label_rating_drop"

# 1) firstly just choose numeric_cols
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# 2) exclude target and all future/label
cols_to_exclude = [
    target_col,
    "rating_7d_future",
    "future_rating_14d",
    "rating_drop",
]
cols_to_exclude = [c for c in cols_to_exclude if c in numeric_cols]

feature_cols = [c for c in numeric_cols if c not in cols_to_exclude]

print(f"Total numeric features used: {len(feature_cols)}")
print(feature_cols)

# 3) df X / y，remove  NaN
X = df[feature_cols].copy()
y = df[target_col].copy()

before = X.shape[0]
valid_mask = X.notna().all(axis=1) & y.notna()
X = X[valid_mask]
y = y[valid_mask]
print(f"Removed {before - X.shape[0]} rows due to NaNs in features/label.")

# 4) Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape :", X_test.shape)

# 5) Check class balance and apply SMOTE only if needed
from collections import Counter

class_counts = Counter(y_train)
minority_class = min(class_counts, key=class_counts.get)
majority_class = max(class_counts, key=class_counts.get)

minority_count = class_counts[minority_class]
majority_count = class_counts[majority_class]
current_ratio = minority_count / majority_count

desired_ratio = 0.3

print("\nClass distribution before SMOTE:")
print(class_counts)
print(f"Current minority/majority ratio: {current_ratio:.3f}")

if current_ratio >= desired_ratio:
    print(
        f"Current ratio ({current_ratio:.3f}) >= desired ratio ({desired_ratio}), skipping SMOTE."
    )
    X_train_balanced = X_train.copy()
    y_train_balanced = y_train.copy()
else:
    print(f"Applying SMOTE with sampling_strategy={desired_ratio} ...")
    smote = SMOTE(sampling_strategy=desired_ratio, random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("\nAfter SMOTE (or skip):")
print("X_train_balanced:", X_train_balanced.shape)
print("y_train_balanced distribution (proportion):")
print(y_train_balanced.value_counts(normalize=True))


Total numeric features used: 29
['stars_review', 'useful', 'funny', 'cool', 'stars_business', 'text_length', 'sentiment', 'sentiment_7d', 'sentiment_14d', 'rating_7d', 'rating_14d', 'review_count_7d', 'review_count_14d', 'sentiment_momentum', 'rating_momentum', 'sentiment_slope', 'rating_volatility', 'rating_sentiment_interaction', 'momentum_volatility', 'day_of_week', 'month', 'is_weekend', 'is_holiday_season', 'review_density', 'negative_ratio_7d', 'rating_range_7d', 'sentiment_range_7d', 'sentiment_acceleration', 'rating_acceleration']
Removed 13964 rows due to NaNs in features/label.
Train shape: (60731, 29)
Test shape : (15183, 29)

Class distribution before SMOTE:
Counter({0: 47349, 1: 13382})
Current minority/majority ratio: 0.283
Applying SMOTE with sampling_strategy=0.3 ...

After SMOTE (or skip):
X_train_balanced: (61553, 29)
y_train_balanced distribution (proportion):
label_rating_drop
0    0.76924
1    0.23076
Name: proportion, dtype: float64


## 5. Optimized Random Forest Training

We train a Random Forest model on the SMOTE-balanced training data.
Hyperparameters are chosen to balance performance and interpretability.


In [ ]:
rf_params = {
    "n_estimators": 400,
    "max_depth": 8,
    "min_samples_split": 50,
    "min_samples_leaf": 20,
    "max_features": "sqrt",
    "random_state": 42,
    "n_jobs": -1,
    "class_weight": None,
}

model = RandomForestClassifier(**rf_params)
model.fit(X_train_balanced, y_train_balanced)

print("Random Forest trained with optimized settings.")


Random Forest trained with optimized settings.


## 6. Baseline Time-Series Model (Logistic Regression)

To provide a meaningful **baseline**, we train a simple logistic regression
model using only a subset of time-series features.


In [ ]:
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.filterwarnings("ignore", category=ConvergenceWarning)

ts_features = [
    "rating_7d",
    "sentiment_7d",
    "rating_momentum",
    "sentiment_momentum",
    "rating_volatility",
    "sentiment_volatility",
    "review_count_7d",
]
ts_features = [f for f in ts_features if f in X_train.columns]

X_train_ts = X_train[ts_features]
X_test_ts = X_test[ts_features]

ts_model = LogisticRegression(max_iter=500, class_weight="balanced")
ts_model.fit(X_train_ts, y_train)

y_proba_ts = ts_model.predict_proba(X_test_ts)[:, 1]
y_pred_ts = (y_proba_ts >= 0.5).astype(int)


def print_metrics(prefix, y_true, y_proba, y_pred):
    print(prefix)
    print(f"  AUC:       {roc_auc_score(y_true, y_proba):.3f}")
    print(f"  Accuracy:  {accuracy_score(y_true, y_pred):.3f}")
    print(f"  Precision: {precision_score(y_true, y_pred):.3f}")
    print(f"  Recall:    {recall_score(y_true, y_pred):.3f}")
    print(f"  F1-score:  {f1_score(y_true, y_pred):.3f}")


print_metrics(
    "\nTime-series baseline (Logistic Regression)", y_test, y_proba_ts, y_pred_ts
)



Time-series baseline (Logistic Regression)
  AUC:       0.679
  Accuracy:  0.624
  Precision: 0.324
  Recall:    0.647
  F1-score:  0.432


## 7. Threshold Optimization for Random Forest

We compute predicted probabilities from the optimized Random Forest, and
search over thresholds to find the one that maximizes F1-score.


In [ ]:
# Predicted probabilities from optimized RF
y_proba_optimized = model.predict_proba(X_test)[:, 1]

# Search thresholds
thresholds = np.linspace(0.1, 0.9, 81)
f1_scores = []
for t in thresholds:
    y_pred_t = (y_proba_optimized >= t).astype(int)
    f1_scores.append(f1_score(y_test, y_pred_t))

best_idx = int(np.argmax(f1_scores))
best_threshold = float(thresholds[best_idx])
print(f"Best threshold by F1-score: {best_threshold:.3f}")
print(f"Best F1-score: {f1_scores[best_idx]:.3f}")

# Final predictions at optimal threshold
y_pred_optimized = (y_proba_optimized >= best_threshold).astype(int)

print_metrics(
    "\nRandom Forest (Optimized, best threshold)",
    y_test,
    y_proba_optimized,
    y_pred_optimized,
)

# Plot F1 vs threshold
plt.figure()
plt.plot(thresholds, f1_scores, marker=".")
plt.axvline(
    best_threshold, color="red", linestyle="--", label=f"Best: {best_threshold:.3f}"
)
plt.xlabel("Threshold")
plt.ylabel("F1-score")
plt.title("Threshold Optimization (Random Forest)")
plt.legend()
plt.tight_layout()
plt.savefig("figures/threshold_optimization.png", dpi=150)
plt.close()
print("✓ Saved: figures/threshold_optimization.png")


Best threshold by F1-score: 0.250
Best F1-score: 0.495

Random Forest (Optimized, best threshold)
  AUC:       0.753
  Accuracy:  0.703
  Precision: 0.396
  Recall:    0.661
  F1-score:  0.495
✓ Saved: figures/threshold_optimization.png


## 8. Model Selection & Benchmark Comparison

We compare several models:

- Logistic Regression (time-series only)
- Logistic Regression (all features)
- Gradient Boosting
- Random Forest (Optimized)


In [ ]:
results = []


def eval_model(name, y_true, y_proba, y_pred):
    return {
        "model": name,
        "auc": roc_auc_score(y_true, y_proba),
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
    }


# 1) Baseline TS logistic
results.append(eval_model("Logistic (TS only)", y_test, y_proba_ts, y_pred_ts))

# 2) Logistic Regression (all features)
log_all = LogisticRegression(max_iter=500, class_weight="balanced")
log_all.fit(X_train, y_train)
y_proba_log_all = log_all.predict_proba(X_test)[:, 1]
y_pred_log_all = (y_proba_log_all >= 0.5).astype(int)
results.append(
    eval_model("Logistic (All features)", y_test, y_proba_log_all, y_pred_log_all)
)

# 3) Gradient Boosting
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
y_proba_gb = gb.predict_proba(X_test)[:, 1]
y_pred_gb = (y_proba_gb >= 0.5).astype(int)
results.append(eval_model("Gradient Boosting", y_test, y_proba_gb, y_pred_gb))

# 4) Optimized Random Forest
results.append(
    eval_model("Random Forest (Optimized)", y_test, y_proba_optimized, y_pred_optimized)
)

comparison_df = pd.DataFrame(results).sort_values("auc", ascending=False)
print("\nMODEL COMPARISON SUMMARY:")
print(comparison_df.to_string(index=False, float_format=lambda x: f"{x:.3f}"))

comparison_df.to_csv("models/model_comparison_summary.csv", index=False)
print("\n✓ Saved: models/model_comparison_summary.csv")



MODEL COMPARISON SUMMARY:
                    model   auc  accuracy  precision  recall    f1
        Gradient Boosting 0.770     0.792      0.597   0.170 0.264
  Logistic (All features) 0.767     0.684      0.383   0.714 0.499
Random Forest (Optimized) 0.753     0.703      0.396   0.661 0.495
       Logistic (TS only) 0.679     0.624      0.324   0.647 0.432

✓ Saved: models/model_comparison_summary.csv


## 9. Feature Importance (Random Forest)

We compute and visualize feature importances from the optimized Random Forest.


In [ ]:
importances = pd.Series(model.feature_importances_, index=feature_cols).sort_values(
    ascending=False
)
top_n = 20
top_importances = importances.head(top_n)

plt.figure(figsize=(8, 6))
top_importances[::-1].plot(kind="barh")
plt.xlabel("Importance")
plt.title(f"Top {top_n} Feature Importances (Random Forest)")
plt.tight_layout()
plt.savefig("figures/model_results_optimized.png", dpi=150)
plt.close()
print("✓ Saved: figures/model_results_optimized.png")


✓ Saved: figures/model_results_optimized.png


## 10. SHAP Explainability

We apply SHAP (Shapley Additive Explanations) to interpret the optimized
Random Forest at both global and local levels.


In [ ]:
print("\nComputing SHAP values (this may take some time)...")

sample_size = min(2000, len(X_test))
X_sample = X_test.sample(sample_size, random_state=42)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_sample)
shap_pos = shap_values[1] if isinstance(shap_values, list) else shap_values

# Summary plot
shap.summary_plot(shap_pos, X_sample, show=False)
plt.tight_layout()
plt.savefig("figures/shap_summary_optimized_rf.png", dpi=150)
plt.close()
print("Saved: figures/shap_summary_optimized_rf.png")



Computing SHAP values (this may take some time)...
Saved: figures/shap_summary_optimized_rf.png


## 11. Save Model Artifacts and Optimized Feature Dataset


In [ ]:
import pickle

# Save model
model_path = "models/rating_drop_model_optimized.pkl"
with open(model_path, "wb") as f:
    pickle.dump(model, f)
print("✓ Saved model to:", model_path)

# Save feature list
with open("models/feature_list_optimized.txt", "w") as f:
    for col in feature_cols:
        f.write(col + "\n")
print("✓ Saved feature list to: models/feature_list_optimized.txt")

# Save model summary
summary = {
    "model": "Random Forest (Optimized)",
    "n_features": len(feature_cols),
    "n_original_features": None,  # can fill manually if needed
    "n_new_features": None,  # can fill manually if needed
    "roc_auc": roc_auc_score(y_test, y_proba_optimized),
    "accuracy": accuracy_score(y_test, y_pred_optimized),
    "precision": precision_score(y_test, y_pred_optimized),
    "recall": recall_score(y_test, y_pred_optimized),
    "f1_score": f1_score(y_test, y_pred_optimized),
    "optimal_threshold": best_threshold,
    "train_samples": len(X_train),
    "train_samples_balanced": len(X_train_balanced),
    "test_samples": len(X_test),
    "smote_applied": True,
    "smote_sampling_strategy": 0.3,
}

summary_df = pd.DataFrame([summary])
summary_df.to_csv("models/model_summary_optimized.csv", index=False)
print("✓ Saved: models/model_summary_optimized.csv")

# Save optimized features dataset
optimized_path = "data/processed/reviews_features_optimized.csv"
df.to_csv(optimized_path, index=False)
print("✓ Saved optimized features to:", optimized_path)


✓ Saved model to: models/rating_drop_model_optimized.pkl
✓ Saved feature list to: models/feature_list_optimized.txt
✓ Saved: models/model_summary_optimized.csv
✓ Saved optimized features to: data/processed/reviews_features_optimized.csv


## 12. Run Summary


In [ ]:
print("\n" + "=" * 70)
print("MODEL OPTIMIZATION COMPLETE")
print("=" * 70)

print("\nFinal Model Performance (Random Forest, Optimized):")
print(f"  ROC AUC:   {roc_auc_score(y_test, y_proba_optimized):.3f}")
print(f"  Accuracy:  {accuracy_score(y_test, y_pred_optimized):.3f}")
print(f"  Precision: {precision_score(y_test, y_pred_optimized):.3f}")
print(f"  Recall:    {recall_score(y_test, y_pred_optimized):.3f}")
print(f"  F1-Score:  {f1_score(y_test, y_pred_optimized):.3f}")
print(f"  Best threshold: {best_threshold:.3f}")

print("\nArtifacts saved:")
print("  - models/rating_drop_model_optimized.pkl")
print("  - models/model_summary_optimized.csv")
print("  - models/feature_list_optimized.txt")
print("  - models/model_comparison_summary.csv")
print("  - figures/model_results_optimized.png")
print("  - figures/threshold_optimization.png")
print("  - figures/shap_summary_optimized_rf.png")
print("  - figures/shap_single_example_optimized_rf.png")
print("  - data/processed/reviews_features_optimized.csv")



MODEL OPTIMIZATION COMPLETE

Final Model Performance (Random Forest, Optimized):
  ROC AUC:   0.753
  Accuracy:  0.703
  Precision: 0.396
  Recall:    0.661
  F1-Score:  0.495
  Best threshold: 0.250

Artifacts saved:
  - models/rating_drop_model_optimized.pkl
  - models/model_summary_optimized.csv
  - models/feature_list_optimized.txt
  - models/model_comparison_summary.csv
  - figures/model_results_optimized.png
  - figures/threshold_optimization.png
  - figures/shap_summary_optimized_rf.png
  - figures/shap_single_example_optimized_rf.png
  - data/processed/reviews_features_optimized.csv
